In [2]:
# import libraries

from bs4 import BeautifulSoup # For HTML parsing
import urllib # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
%matplotlib inline


In [4]:
#Build search criteria

search_item = 'condo' 

base_url = 'https://toronto.craigslist.ca/'

final_site_list = ['https://toronto.craigslist.ca/search/hhh?query=', search_item, '&sort=rel'] 
# Join all of our strings together 
    
final_site = ''.join(final_site_list)

print("----- Print the URL component as list -----")
print(final_site_list)
print("")
print("----- Print the final concatenated URL -----")
print(final_site)

----- Print the URL component as list -----
['https://toronto.craigslist.ca/search/hhh?query=', 'condo', '&sort=rel']

----- Print the final concatenated URL -----
https://toronto.craigslist.ca/search/hhh?query=condo&sort=rel


In [ ]:
###### Part 1 - main page scraping #######

In [5]:
# Download and parse the page

# read the URL content
html = urllib.request.urlopen(final_site).read() # Open up the front page of our search first

# create a beautifulsoup object
soup = BeautifulSoup(html, 'html5lib') # Get the html from the first page

print(soup)

<!DOCTYPE html>
<html class="no-js"><head>
    <title>toronto housing "condo" - craigslist</title>

    <meta content='toronto housing "condo" - craigslist' name="description"/>
    <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
    <link href="https://toronto.craigslist.ca/search/hhh" rel="canonical"/>
    <link href="https://toronto.craigslist.ca/search/hhh?format=rss&amp;query=condo&amp;sort=rel" rel="alternate" title='RSS feed for craigslist | toronto housing "condo" - craigslist' type="application/rss+xml"/>
        <link href="https://toronto.craigslist.ca/search/hhh?s=120&amp;query=condo&amp;sort=rel" rel="next"/>
    <meta content="width=device-width,initial-scale=1" name="viewport"/>
    <link href="//www.craigslist.org/styles/cl.css?v=58db8b9d287844a7f120e277e3e8439c" media="all" rel="stylesheet" type="text/css"/>
    <link href="//www.craigslist.org/styles/search.css?v=27e1d4246df60da5ffd1146d59a8107e" media="all" rel="stylesheet" type="text/css"/>
    <link href="//

In [10]:
# the section which contain result info (main page)

listing_area_all_id = soup.find_all(class_ = 'result-info') 
print(listing_area_all_id)

[<p class="result-info">
        <span class="icon icon-star" role="button">
            <span class="screen-reader-text">favorite this post</span>
        </span>

            <time class="result-date" datetime="2017-12-20 09:49" title="Wed 20 Dec 09:49:13 AM">Dec 20</time>


        <a class="result-title hdrlnk" data-id="6429097973" href="https://toronto.craigslist.ca/tor/apa/d/immediate-move-in-trendy/6429097973.html">Immediate Move In Trendy Junior 1 Bedroom Loft Style Feel In King West</a>


        <span class="result-meta">
                <span class="result-price">$1900</span>

                <span class="housing">
                    1br -
                    471ft<sup>2</sup> -
                </span>

                <span class="result-hood"> (533 Richmond St W)</span>

                <span class="result-tags">
                    pic
                    <span class="maptag" data-pid="6429097973">map</span>
                </span>

                <span class="banish ic

In [12]:
# extract the id

listing_result_info_0a = [linkid.find('a') for linkid in listing_area_all_id]  # note: do NOT make this a string, otherwise 'str' object has no attribute 'get'
listing_result_info_0a

[<a class="result-title hdrlnk" data-id="6429097973" href="https://toronto.craigslist.ca/tor/apa/d/immediate-move-in-trendy/6429097973.html">Immediate Move In Trendy Junior 1 Bedroom Loft Style Feel In King West</a>,
 <a class="result-title hdrlnk" data-id="6429098176" href="https://toronto.craigslist.ca/tor/apa/d/move-in-now-library-district/6429098176.html">Move In Now @ Library District Junior 1 Bedroom With Locker</a>,
 <a class="result-title hdrlnk" data-id="6429098396" href="https://toronto.craigslist.ca/tor/apa/d/fabulous-bloor-11-2-bedroom/6429098396.html">Fabulous Bloor E. 1+1/ 2 Bedroom Close To Transit &amp; Restaurants</a>,
 <a class="result-title hdrlnk" data-id="6429098697" href="https://toronto.craigslist.ca/tor/apa/d/harbour-plaza-2-bedroom-2/6429098697.html">Harbour Plaza 2 Bedroom 2 Bath W. Locker Condo Ready For Quick Move In</a>,
 <a class="result-title hdrlnk" data-id="6429099554" href="https://toronto.craigslist.ca/tor/apa/d/studio-waterfront-view-path/6429099554.

In [15]:
listing_result_info_0b = [str(linkid.get('data-id')) for linkid in listing_result_info_0a]
listing_result_info_0b[0:10]

['6429097973',
 '6429098176',
 '6429098396',
 '6429098697',
 '6429099554',
 '6432325506',
 '6432321550',
 '6432322085',
 '6432322637',
 '6432320503']

In [16]:
len(listing_result_info_0b)

120

In [21]:
# extract the price

listing_result_info_11 = []

for linkid in listing_area_all_id:

    try:
        text = linkid.find(class_ = 'result-price').string
    except Exception:
        text = 'No Data' 

    listing_result_info_11.append(text)

In [22]:
listing_result_info_11[0:10]

['$1900',
 '$1750',
 '$2250',
 '$2500',
 '$1800',
 '$558888',
 '$4295',
 '$4295',
 '$2595',
 '$554900']

In [23]:
len(listing_result_info_11)

120

In [24]:
# extract the price 

listing_result_info_12 = []

for linkid in listing_area_all_id:

    try:
        text = linkid.find(class_ = 'housing').get_text()
    except Exception:
        text = 'No Data' 

    listing_result_info_12.append(text)

In [25]:
listing_result_info_12[0:10]

['\n                    1br -\n                    471ft2 -\n                ',
 '\n                    1br -\n                ',
 '\n                    2br -\n                    724ft2 -\n                ',
 '\n                    2br -\n                    675ft2 -\n                ',
 'No Data',
 '\n                    1br -\n                    1500ft2 -\n                ',
 '\n                    1br -\n                    638ft2 -\n                ',
 '\n                    1br -\n                    700ft2 -\n                ',
 '\n                    1br -\n                    702ft2 -\n                ',
 '\n                    1br -\n                    550ft2 -\n                ']

In [26]:
len(listing_result_info_12)

120

In [ ]:
# extract the address

In [27]:
listing_result_info_13 = []

for linkid in listing_area_all_id:

    try:
        text = linkid.find(class_ = 'result-hood').string
    except Exception:
        text = 'No Data' 

    listing_result_info_13.append(text)

In [28]:
listing_result_info_13[0:10]

[' (533 Richmond St W)',
 ' (170 Fort York Blvd)',
 ' (101 Charles St E)',
 ' (100 Harbour St)',
 ' (100 Harbour St)',
 ' (565 Wilson Ave)',
 ' (Yonge/Davenport)',
 ' (King/John)',
 ' (St. Lawrence Market)',
 ' (14 York St)']

In [29]:
len(listing_result_info_13)

120

In [30]:
import numpy as np
df_4_col = pd.DataFrame(np.column_stack([listing_result_info_0b, listing_result_info_11, listing_result_info_12, listing_result_info_13]), 
                               columns=['URL_ID', 'Price', 'Layout','Address'])
df_4_col

,URL_ID,Price,Layout,Address
0,6429097973,$1900,\n 1br -\n ...,(533 Richmond St W)
1,6429098176,$1750,\n 1br -\n,(170 Fort York Blvd)
2,6429098396,$2250,\n 2br -\n ...,(101 Charles St E)
3,6429098697,$2500,\n 2br -\n ...,(100 Harbour St)
4,6429099554,$1800,No Data,(100 Harbour St)
5,6432325506,$558888,\n 1br -\n ...,(565 Wilson Ave)
6,6432321550,$4295,\n 1br -\n ...,(Yonge/Davenport)
7,6432322085,$4295,\n 1br -\n ...,(King/John)
8,6432322637,$2595,\n 1br -\n ...,(St. Lawrence Market)
9,6432320503,$554900,\n 1br -\n ...,(14 York St)


In [ ]:
###### Part 2 - detailed page scraping #######

In [36]:
listing_URLS = [str(linkid.get('href')) for linkid in listing_result_info_0a]
listing_URLS[0:10]

['https://toronto.craigslist.ca/tor/apa/d/immediate-move-in-trendy/6429097973.html',
 'https://toronto.craigslist.ca/tor/apa/d/move-in-now-library-district/6429098176.html',
 'https://toronto.craigslist.ca/tor/apa/d/fabulous-bloor-11-2-bedroom/6429098396.html',
 'https://toronto.craigslist.ca/tor/apa/d/harbour-plaza-2-bedroom-2/6429098697.html',
 'https://toronto.craigslist.ca/tor/apa/d/studio-waterfront-view-path/6429099554.html',
 'https://toronto.craigslist.ca/tor/reb/d/gorgeous-1-bdrm-den-condo/6432325506.html',
 'https://toronto.craigslist.ca/tor/apa/d/exciting-1-bedroom-suite-with/6432321550.html',
 'https://toronto.craigslist.ca/tor/apa/d/luxurious-1-bedroom-corner/6432322085.html',
 'https://toronto.craigslist.ca/tor/apa/d/stylish-1-bedroom-2-bathroom/6432322637.html',
 'https://toronto.craigslist.ca/tor/reb/d/amazing-1-bdrm-condo/6432320503.html']

In [37]:
len(listing_URLS)

120

In [39]:
# total number of condos listing
num_listing_area = soup.find(class_ = 'totalcount').string
num_listing_area

'2500'

In [40]:
total_num_listing = int(num_listing_area)
total_num_listing

2500

In [41]:
# number of pages
num_pages = total_num_listing // 120
num_pages

20

In [42]:
# Parse the URL
from urllib.parse import urlparse 
final_site

'https://toronto.craigslist.ca/search/hhh?query=condo&sort=rel'

In [43]:
final_site_split = urlparse(final_site)
final_site_split

ParseResult(scheme='https', netloc='toronto.craigslist.ca', path='/search/hhh', params='', query='query=condo&sort=rel', fragment='')

In [45]:
# function to cleanup 
def text_cleaner(website):
    '''
    This function just cleans up the raw html so that I can look at it.
    Inputs: a URL to investigate
    Outputs: Cleaned text only
    '''
    try:
        site = urllib.request.urlopen(website).read() # Connect to the posting
    except: 
        return   # Need this in case the website isn't there anymore or some other weird connection problem 

    soup_obj = BeautifulSoup(site, 'html5lib') # Get the html from the site

    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object

    text = soup_obj.get_text() # Get the text from this
    lines = (line.strip() for line in text.splitlines()) # break into lines
    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each

    def chunk_space(chunk):
        chunk_out = chunk + ' ' # Need to fix spacing issue
        return chunk_out  

    text = ''.join(chunk_space(chunk) for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line

    # Now clean out all of the unicode junk (this line works great!!!)
    try:
        text = text.decode('unicode_escape') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception

    text = re.sub("[^a-zA-Z.+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                                # Also include + for C++

    #text = text.lower().split()  # Go to lower case and split them apart
    #stop_words = set(stopwords.words("english")) # Filter out any stop words
    #text = [w for w in text if not w in stop_words]
    #text = list(set(text)) # Last, just get the set of these. Ignore counts (we are just looking at whether a term existed
                            # or not on the website)
    return text

In [47]:
listing_descriptions = [] # Store all our descriptions in this list

In [49]:
    for i in range(1,2): # Loop through all of our search result pages
        print('Getting page', i)
        
        start_num = str(i*120) # Assign the multiplier to view the pages we want (in my case multiplier is 120)
        
        current_page = ''.join([final_site_split.scheme,'://',final_site_split.netloc ,final_site_split.path ,'?','s=', start_num, '&', final_site_split.query])
        # Now that we can view the correct 120 listing returns, start collecting the text samples from each

        html_page = urllib.request.urlopen(current_page).read() # Get the page

        page_obj = BeautifulSoup(html_page, 'lxml') # Locate all of the listing links
        
        listing_area_all_id = page_obj.find_all(class_ = 'result-info') 
        
        listing_result_info_0a = [linkid.find('a') for linkid in listing_area_all_id]
        
        listing_URLS = [str(linkid.get('href')) for linkid in listing_result_info_0a]

        for j in range(0,len(listing_URLS)):
            final_description = text_cleaner(listing_URLS[j])
            if final_description: # only append when the website was accessed correctly
                listing_descriptions.append(final_description)
            sleep(1) # So that we don't be jerks. If you have a very fast internet connection you could hit the server a lot! 

    print('Done with collecting the job postings!')    
    print('There were' + str(len(listing_descriptions)) + 'listing successfully found.')

Getting page 1
Done with collecting the job postings!
There were120listing successfully found.


In [50]:
listing_descriptions

['Modern   Bedroom Suite wht Parking and Central Location   Furnished   apts housing for rent CL toronto   city of toronto   housing   apts housing for rent post account favorites hidden CL city of toronto   apts housing for rent ...     prev     next     reply         reply x prohibited    Posted          3   3 am Contact Information  print favorite this post  3       br      ft    Modern   Bedroom Suite wht Parking and Central Location   Furnished  King Spadina  hide this posting unhide   image   of       google map   BR    Ba    ft  available dec   cats are OK   purrr dogs are OK   wooof furnished condo w d in unit attached garage QR Code Link to This Post FURNISHED     BED     BATH   CONDO   APARTMENT       SQ. FT. Financial District Toronto Furnished Condo Rental at Reve Condos   Modern   Bedroom Suite wht Parking and Central Location  3      month Available  Immediately KING WEST VILLAGE     Front Street West       Toronto Cozy yet modern  this Financial District Toronto furnishe

In [51]:

df_1_col = pd.DataFrame(np.column_stack([listing_descriptions]), 
                               columns=['Descriptions'])
df_1_col

,Descriptions
0,Modern Bedroom Suite wht Parking and Central...
1,Gorgeous Bedroom Suite with Stunning Views a...
2,Luxurious Bed Bath Suite with Wrap Around ...
3,Contemporary Bedroom Bath Corner Suite wit...
4,Beautiful Bed Suite with French Doors Grea...
5,Modern Bedroom Suite with Balcony and Downto...
6,Completely Renovated 3 Bdrm Condo Sun Filled ...
7,+den sf high floor unobstructed view ...
8,Prestigious Bdrm Soho Met Condo In Amazing L...
9,Thompson Hotel Penthouse One Bedroom a...
